# Jigsaw Puzzle — Milestone 1 Pipeline

This notebook implements a full Milestone-1 pipeline for the puzzle dataset located at `/mnt/data/1.jpg`.

**Features implemented:**
- Automatic grid detection (projection profiles) with fallback to manual grid size
- Tile extraction for 2x2, 4x4, 8x8 puzzles
- Per-tile enhancement (denoise, CLAHE, optional resize)
- Binary mask creation and contour extraction
- Artifacts saving (enhanced images, masks, contours, metadata)
- Visualization helpers (before/after and grid overlay)

Run each cell in order. Modify parameters near the top if needed.


In [ ]:

# Imports and parameters
import cv2, os, json, numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

INPUT_PATH = "D:/Image-Processing-Project/Gravity_Falls/puzzle_2x2/1.jpg"   # representative image
OUT_DIR = "D:/Image-Processing-Project/output"
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUT_DIR,"tiles"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR,"visualizations"), exist_ok=True)

# Parameters (tweak if needed)
FALLBACK_GRID = None   # set to (rows,cols) if auto-detect fails, e.g. (4,4)
PADDING = 4            # pixels to include around tiles when cropping
RESIZE_TO = None       # set e.g. (256,256) to resize tiles for uniformity, or None to keep original
SAVE_CONTOUR_NPY = True


In [ ]:
import cv2
import numpy as np
import os


def detect_salt_noise(img, bright_thresh=220, ratio=0.001):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    salt_ratio = np.sum(gray > bright_thresh) / gray.size
    return salt_ratio > ratio, salt_ratio


def detect_pepper_noise_median(img, dark_thresh=60, med_kernel=3, diff_threshold=30, ratio=0.001):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    median = cv2.medianBlur(gray, med_kernel)

    pepper_mask = np.logical_and(gray < dark_thresh, (median - gray) >= diff_threshold)
    ratio_median = np.sum(pepper_mask) / gray.size

    return (ratio_median > ratio, ratio_median)


def detect_gaussian_noise(img, threshold=15):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    noise_level = gray.std()
    return noise_level > threshold


def detect_blur(img, threshold=120):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm < threshold


def enhance_image(img):
    print("\n===== IMAGE ANALYSIS =====")
    
    has_salt, salt_ratio = detect_salt_noise(img)
    has_pepper, pepper_ratio = detect_pepper_noise_median(img)
    has_gaussian = detect_gaussian_noise(img, threshold=50) 
    is_blurry = detect_blur(img, threshold=80)               

    print(f"Salt noise detected: {has_salt}  (ratio={salt_ratio:.5f})")
    print(f"Pepper noise detected: {has_pepper} (ratio={pepper_ratio:.5f})")
    print(f"Gaussian noise detected: {has_gaussian}")
    print(f"Image is blurry: {is_blurry}\n")


    if (
        not has_salt and salt_ratio < 0.0005 and
        not has_pepper and pepper_ratio < 0.0005 and
        not has_gaussian and
        not is_blurry
    ):
        print("Image is clean. No enhancement applied.")
        return img  

    enhanced = img.copy()

    # Only apply sharpening if blur is significant
    if is_blurry:
        blurred = cv2.GaussianBlur(enhanced, (9, 9), 0)
        enhanced = cv2.addWeighted(enhanced, 1.2, blurred, -0.2, 0)
        print("Sharpening applied.")

    if has_salt or has_pepper:
        enhanced = cv2.medianBlur(enhanced, 9)

    if has_gaussian:
        enhanced = cv2.bilateralFilter(enhanced, 9, 75, 75)

    return enhanced


if __name__ == "__main__":

    image_path = r"C:\Users\nada\Downloads\Image-Processing-Project\Gravity_Falls\nnn.png"
    save_path  = r"C:\Users\nada\Downloads\Image-Processing-Project\data_enhanced\puzzle1_enhanced.jpg"

    img = cv2.imread(image_path)

    if img is None:
        print("Error: Image not found")
    else:
        enhanced = enhance_image(img)

        scale = 0.4
        img_small = cv2.resize(img, None, fx=scale, fy=scale)
        enh_small = cv2.resize(enhanced, None, fx=scale, fy=scale)

        combined = np.hstack((img_small, enh_small))
        cv2.imshow("Original | Enhanced", combined)

        diff = cv2.absdiff(img_small, enh_small)
        cv2.imshow("Difference", diff)

        cv2.waitKey(0)
        cv2.destroyAllWindows()

        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        cv2.imwrite(save_path, enhanced)
        print(f"Enhanced image saved at: {save_path}")



===== IMAGE ANALYSIS =====
Salt noise detected: True  (ratio=0.02391)
Pepper noise detected: True (ratio=0.00745)
Gaussian noise detected: False
Image is blurry: False

Enhanced image saved at: C:\Users\nada\Downloads\Image-Processing-Project\data_enhanced\puzzle1_enhanced.jpg
